# Import Statements

In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle

# Machine Learning

**Data has been preprocessed**


In [23]:
data = pd.read_csv('Absenteeism_preprocessed.csv')
data.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month,Weekday,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,False,False,False,True,7,1,289,36,33,239.554,30,0,2,1,4
1,False,False,False,False,7,1,118,13,50,239.554,31,0,1,0,0
2,False,False,False,True,7,2,179,51,38,239.554,31,0,0,0,2
3,True,False,False,False,7,3,279,5,39,239.554,24,0,2,0,4
4,False,False,False,True,7,3,289,36,33,239.554,30,0,2,1,2


### Targets

In [24]:
# Create Excessive absenteeism column representing absence of over 3 hours
targets = np.where(data['Absenteeism Time in Hours'] >
                   data['Absenteeism Time in Hours'].median(), 1, 0)
data['Excessive Absenteeism'] = targets
data.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month,Weekday,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteeism
0,False,False,False,True,7,1,289,36,33,239.554,30,0,2,1,4,1
1,False,False,False,False,7,1,118,13,50,239.554,31,0,1,0,0,0
2,False,False,False,True,7,2,179,51,38,239.554,31,0,0,0,2,0
3,True,False,False,False,7,3,279,5,39,239.554,24,0,2,0,4,1
4,False,False,False,True,7,3,289,36,33,239.554,30,0,2,1,2,0


In [25]:
# Check for imbalance
targets.sum() / targets.shape[0]

0.45571428571428574

In [26]:
# Drop unnecessary columns
data_with_targets = data.drop(['Absenteeism Time in Hours'],axis=1)
data_with_targets.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month,Weekday,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Excessive Absenteeism
0,False,False,False,True,7,1,289,36,33,239.554,30,0,2,1,1
1,False,False,False,False,7,1,118,13,50,239.554,31,0,1,0,0
2,False,False,False,True,7,2,179,51,38,239.554,31,0,0,0,0
3,True,False,False,False,7,3,279,5,39,239.554,24,0,2,0,1
4,False,False,False,True,7,3,289,36,33,239.554,30,0,2,1,0


In [27]:
data_with_targets.shape

(700, 15)

### Inputs

In [28]:
data_with_targets.iloc[:,:-1]
unscaled_inputs = data_with_targets.iloc[:,:-1]

### Standardization

In [29]:
# Standardize inputs using a custom scaler
class CustomScaler(BaseEstimator,TransformerMixin):
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        self.scaler = StandardScaler(copy=copy, with_mean=with_mean, with_std=with_std)
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        self.copy = copy
        self.with_mean = with_mean
        self.with_std = with_std


    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self


    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [30]:
columns_to_scale = ['Month','Weekday', 'Transportation Expense', 'Distance to Work',
       'Age', 'Daily Work Load Average', 'Body Mass Index', 'Children', 'Pets']

In [31]:
absenteeism_scaler = CustomScaler(columns_to_scale)
absenteeism_scaler.fit(unscaled_inputs)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3785: FutureWarning: The behavior of DataFrame.var with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return var(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


CustomScaler(columns=['Month', 'Weekday', 'Transportation Expense',
                      'Distance to Work', 'Age', 'Daily Work Load Average',
                      'Body Mass Index', 'Children', 'Pets'])

In [32]:
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)
scaled_inputs

,Reason_1,Reason_2,Reason_3,Reason_4,Month,Weekday,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets
0,False,False,False,True,0.182726,-0.683704,1.005844,0.412816,-0.536062,-0.806331,0.767431,0,0.880469,0.268487
1,False,False,False,False,0.182726,-0.683704,-1.574681,-1.141882,2.130803,-0.806331,1.002633,0,-0.019280,-0.589690
2,False,False,False,True,0.182726,-0.007725,-0.654143,1.426749,0.248310,-0.806331,1.002633,0,-0.919030,-0.589690
3,True,False,False,False,0.182726,0.668253,0.854936,-1.682647,0.405184,-0.806331,-0.643782,0,0.880469,-0.589690
4,False,False,False,True,0.182726,0.668253,1.005844,0.412816,-0.536062,-0.806331,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,True,False,False,False,-0.388293,-0.007725,-0.654143,-0.533522,0.562059,-0.853789,-1.114186,1,0.880469,-0.589690
696,True,False,False,False,-0.388293,-0.007725,0.040034,-0.263140,-1.320435,-0.853789,-0.643782,0,-0.019280,1.126663
697,True,False,False,False,-0.388293,0.668253,1.624567,-0.939096,-1.320435,-0.853789,-0.408580,1,-0.919030,-0.589690
698,False,False,False,True,-0.388293,0.668253,0.190942,-0.939096,-0.692937,-0.853789,-0.408580,1,-0.919030,-0.589690


### Logistic Regression

In [33]:
# Split the data
X = scaled_inputs
y = targets
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state = 20)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(560, 14) (560,)
(140, 14) (140,)


In [34]:
# Train the model
reg = LogisticRegression()
reg.fit(x_train,y_train)
reg.score(x_train,y_train)

0.775

In [35]:
reg.intercept_

array([-1.65662792])

In [36]:
reg.coef_

array([[ 2.80136327e+00,  9.33540824e-01,  3.09673857e+00,
         8.57183147e-01,  1.66403124e-01, -8.43159241e-02,
         6.13215559e-01, -7.77871894e-03, -1.65545282e-01,
        -7.68487792e-05,  2.71154773e-01, -2.06026920e-01,
         3.61897667e-01, -2.85728905e-01]])

In [37]:
unscaled_inputs.columns.values
feature_name = unscaled_inputs.columns.values

In [38]:
#Create a summary table with feature names and coefficients
summary_table = pd.DataFrame(columns=['Feature name'],data=feature_name)
summary_table['Coefficient'] = np.transpose(reg.coef_)
summary_table

,Feature name,Coefficient
0,Reason_1,2.801363
1,Reason_2,0.933541
2,Reason_3,3.096739
3,Reason_4,0.857183
4,Month,0.166403
5,Weekday,-0.084316
6,Transportation Expense,0.613216
7,Distance to Work,-0.007779
8,Age,-0.165545
9,Daily Work Load Average,-0.000077


In [39]:
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table['Odds_ratio'] = np.exp(summary_table.Coefficient)
summary_table.sort_values('Odds_ratio', ascending=False)
summary_table

,Feature name,Coefficient,Odds_ratio
0,Intercept,-1.656628,0.190781
1,Reason_1,2.801363,16.467081
2,Reason_2,0.933541,2.543499
3,Reason_3,3.096739,22.125672
4,Reason_4,0.857183,2.356513
5,Month,0.166403,1.181049
6,Weekday,-0.084316,0.919141
7,Transportation Expense,0.613216,1.846359
8,Distance to Work,-0.007779,0.992251
9,Age,-0.165545,0.847431


In [40]:
# Evaluate model performance
reg.score(x_test,y_test)
predicted_proba = reg.predict_proba(x_test)
predicted_proba

array([[0.73844522, 0.26155478],
       [0.60830752, 0.39169248],
       [0.40932165, 0.59067835],
       [0.80500775, 0.19499225],
       [0.07319825, 0.92680175],
       [0.31989141, 0.68010859],
       [0.31315484, 0.68684516],
       [0.1333815 , 0.8666185 ],
       [0.797184  , 0.202816  ],
       [0.75285455, 0.24714545],
       [0.48221987, 0.51778013],
       [0.19628597, 0.80371403],
       [0.07847395, 0.92152605],
       [0.7063866 , 0.2936134 ],
       [0.30657701, 0.69342299],
       [0.57044278, 0.42955722],
       [0.54149644, 0.45850356],
       [0.57202077, 0.42797923],
       [0.38158691, 0.61841309],
       [0.04856239, 0.95143761],
       [0.6977674 , 0.3022326 ],
       [0.79590534, 0.20409466],
       [0.39509601, 0.60490399],
       [0.42263896, 0.57736104],
       [0.26631148, 0.73368852],
       [0.75609167, 0.24390833],
       [0.51061432, 0.48938568],
       [0.86799512, 0.13200488],
       [0.20219374, 0.79780626],
       [0.78649073, 0.21350927],
       [0.

### Save model

In [41]:
with open('model', 'wb') as file:
    pickle.dump(reg, file)

In [42]:
with open('scaler','wb') as file:
    pickle.dump(absenteeism_scaler, file)